In [145]:
from collections import ChainMap
from itertools   import chain
import re
from helpers import data

In [166]:
rules, my_ticket, tickets = data(16, parser=lambda s: s.split("\n"), sep="\n\n")

Create one `tickets` list. 

In [167]:
# Remove text and combine
tickets.pop(0)
my_ticket.pop(0)
# tickets = [*my_ticket, *tickets]
tickets[:3]

# Convert to lists of ints 
tickets = [[int(n) for n in row.split(",")] for row in tickets]

Create one `rules` dictionary. 

In [168]:
rules[:3]

['departure location: 26-715 or 727-972',
 'departure station: 45-164 or 175-960',
 'departure platform: 43-247 or 270-972']

In [169]:
def parse_rule(rule: str) -> dict[str, list[tuple]]:
    """Parse rules into dictionary of ranges.""" 
    d = dict()
    name, rest = rule.split(":")
    d[name] = []
    
    ranges = rest.strip().split(" or ")
    for r in ranges: 
        minimum, maximum = r.split("-")
        d[name].append((int(minimum), int(maximum)))
    
    return d 

assert parse_rule("departure location: 26-715 or 727-972") == {'departure location': [(26, 715), (727, 972)]}

In [170]:
rules = dict(ChainMap(*[d for d in map(parse_rule, rules)]))

**Part 1:** Find the error rate, the sum of invalid values for any field. 

In [171]:
def within(val: int, *bounds: list[tuple[int]]) -> bool:
    """Return whether val is within any of (minimum, maximum) tuples given in bounds."""
    for minimum, maximum in bounds:
        if minimum <= val <= maximum: 
            return True 
    return False

assert within(972, (26, 715), (727, 972))
assert not within(3, (4, 100))

In [178]:
errors = [field for ticket in tickets 
                for field in ticket 
                if not within(field, *list(chain.from_iterable(rules.values())))]

In [179]:
sum(errors)

21081

**Part 1, Improved:**